In [1]:
%pip install openai chromadb==0.3.29 langchain tiktoken
%pip install -U langchain-community
%pip install faiss-cpu
%pip install sentence-transformers

# Import necessary libraries after resolving conflicts and installing dependencies
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from transformers import pipeline
from langchain import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import SimpleSequentialChain
# Import necessary libraries
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain.docstore import InMemoryDocstore
from transformers import pipeline
import numpy as np
import faiss


import os

# List input data files under the input directory
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.2/152.2 kB 5.1 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / - \ | / - done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362.9 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 972.9/972.9 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━



<img src="https://media.licdn.com/dms/image/D4D12AQGkHY89zhGDZA/article-cover_image-shrink_720_1280/0/1691431976579?e=2147483647&v=beta&t=lgk0jF-6kju4RyG68tF_4cxZklJqF3rtXrbQEcEYX7c"></img>

What Are Large **Language Models (LLMs)**?

**Large Language Models (LLMs)** are advanced AI systems built to understand and generate human-like text. Think of them as really smart algorithms trained on vast amounts of written material to grasp language patterns, grammar, and even some factual knowledge. They're often based on the Transformer architecture, which allows them to handle huge datasets efficiently.

What Makes LLMs Special?

Size and Scale:

These models have millions or even billions of parameters. This massive scale enables them to learn a wide range of language features and nuances.

Transformer Architecture:

Most LLMs are built on this architecture, which excels at processing text data in parallel. This makes them both fast and effective at understanding context within large datasets.

Pre-training and Fine-tuning:

LLMs are first pre-trained on a broad set of texts to learn general language patterns. They are then fine-tuned for specific tasks like translation or summarization, making them versatile tools.

Versatility:

They can be used for many tasks: writing articles, translating languages, answering questions, summarizing documents, and even analyzing sentiment. They can handle a wide range of natural language processing (NLP) tasks.

Knowledge Base:

Because they're trained on diverse datasets, LLMs often have a vast reservoir of knowledge. They can provide explanations, answer factual questions, and tackle tasks that require a good grasp of world knowledge.

Popular LLMs You Should Know

GPT (Generative Pre-trained Transformer):

Developed by OpenAI, GPT models are excellent at generating human-like text and have been widely used for applications ranging from chatbots to creative writing.

Examples: GPT-3, GPT-4.

BERT (Bidirectional Encoder Representations from Transformers):

Created by Google, BERT is great for understanding the context of words in a sentence, making it ideal for tasks like question answering.

Examples: BERT, RoBERTa, DistilBERT.

T5 (Text-To-Text Transfer Transformer):
Also from Google, T5 treats every NLP problem as a text generation task, making it highly versatile. It can handle everything from summarization to translation.
Examples: T5, FLAN-T5.

RoBERTa:
A robust, optimized version of BERT from Facebook AI that performs better on various NLP tasks.
XLNet:
Developed by Google and Carnegie Mellon, XLNet builds on BERT by learning context bidirectionally while also being autoregressive, improving its performance.

BLOOM:
A multilingual model from the BigScience project designed to handle multiple languages, trained on diverse text sources.

Where Are LLMs Used?

Chatbots and Virtual Assistants: Making conversations with AI feel more natural.
Content Creation: Writing articles, blog posts, poems, and even code.
Translation: Converting text between languages with high accuracy.
Summarization: Boiling down long documents into concise summaries.
Sentiment Analysis: Detecting emotions or opinions in text.
Question Answering: Providing precise answers to queries, often based on a given context.

Things to Consider with LLMs

Ethical Concerns:
Because they can generate realistic text, LLMs can sometimes be used to spread misinformation or biased content.
Resource Demands:
Training and running these models can be resource-intensive, which makes them less accessible for smaller players.
Interpretability:
It’s often hard to understand why a model gave a particular output, given their complexity.
Bias and Fairness:
LLMs can unintentionally inherit biases from their training data, leading to skewed or unfair outcomes. Ensuring these models are fair and unbiased is an ongoing challenge.
The Bottom Line
LLMs are a huge leap forward in NLP, providing powerful tools for a range of applications. However, with their power comes the responsibility to handle them carefully, especially concerning ethical use, resource needs, and bias. As these models evolve, they'll become even more integral to technology and how we use language.

<img src="https://miro.medium.com/v2/resize:fit:1400/1*NxMFDb5nT1zNqTwRliiNGg.png"></img>

**Simple chatbot using Google model from Huggingface**

In [2]:
from transformers import pipeline
from langchain import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline

# Choose and load a Hugging Face model for text generation
model_name = "google/flan-t5-large"  # Model suitable for text-to-text generation tasks
hf_pipeline = pipeline("text2text-generation", model=model_name, clean_up_tokenization_spaces=True)

# Wrap the Hugging Face pipeline using LangChain's HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=hf_pipeline)

# Create a PromptTemplate
prompt_template = PromptTemplate(input_variables=["question"], template="{question}")

# Create a LangChain LLMChain with the prompt
chain = LLMChain(llm=llm, prompt=prompt_template)

# Run the chain with an input
response = chain.run({"question": "What is the capital of the USA?"})
print(response)


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:151: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(
/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:151: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0

washington


<img src="https://miro.medium.com/v2/resize:fit:1200/1*-PlFCd_VBcALKReO3ZaOEg.png"></img>

**LangChain** is a powerful framework designed for building applications that interact with large language models (LLMs) more effectively. It provides tools to connect LLMs with various data sources, APIs, and other modules, enabling the creation of more dynamic and context-aware applications.

Key Features of LangChain:
**Modularity**: Allows easy integration of different components like prompts, memory, and chains to build customized workflows.

**Prompt Engineering**: Provides tools for crafting and managing prompts to get more precise and relevant outputs from LLMs.
Memory: Enables context retention across interactions, making applications more interactive and capable of handling multi-turn conversations.

**Chain Logic**: Facilitates chaining multiple operations together, such as data retrieval, processing, and response generation, to handle complex tasks.
Applications:

**Conversational Agents**: Build more interactive and context-aware chatbots.

**Document Q&A**: Connect LLMs with external documents to answer questions based on specific content.

**Automation Workflows**: Automate tasks by combining LLM outputs with other tools and APIs.

LangChain simplifies the process of building advanced LLM-driven applications by providing a robust framework that integrates various functionalities seamlessly.

In [3]:

# Create Sample Text Documents as LangChain Documents
sample_texts = [
    "Washington, D.C., formally the District of Columbia, is the capital city of the United States. It is located on the east bank of the Potomac River and was founded after the American Revolution to serve as the nation's capital.",
    "The United States of America (USA) is a country primarily located in North America. It consists of 50 states, a federal district, and several territories.",
    "San Francisco is a major city in California, known for its Golden Gate Bridge, cable cars, and colorful Victorian houses."
]

# Convert sample texts into LangChain Document objects
documents = [Document(page_content=text) for text in sample_texts]

# Chunk Documents
# Split documents into chunks for better retrieval performance
text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=50)
chunked_documents = text_splitter.split_documents(documents)

# Index the Documents
# Use Hugging Face embeddings for indexing
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create an index using FAISS for efficient similarity search
index = FAISS.from_documents(chunked_documents, embedding_model)

# Set Up the RAG Pipeline
# Choose and load a Hugging Face model for text generation
model_name = "google/flan-t5-large"  # Model suitable for text-to-text generation tasks
hf_pipeline = pipeline("text2text-generation", model=model_name)

# Wrap the Hugging Face pipeline using LangChain's HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=hf_pipeline)

# Create a LangChain RetrievalQA chain that combines document retrieval with text generation
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # 'stuff' or 'map_reduce' 
    retriever=index.as_retriever()
)

#Run the RAG Chain with an Input Question
response = rag_chain.run({"query": "What is the capital of the USA?"})
print(response)

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:151: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Washington, D.C.


<img src="https://admin.bentoml.com/uploads/simple_rag_workflow_091648ef39.png"></img>

**Retrieval-Augmented Generation (RAG)** is a clever way to get better answers from AI models by mixing document search with text generation. Here’s the idea:

What RAG Does
Finds Relevant Information: First, it searches a large database of documents to find the most relevant pieces of text related to a question. Think of it like using Google to find articles or passages that are closely related to what you're asking about.

Generates Answers: Once it has those relevant texts, a language model (like GPT or T5) steps in to read through those documents and generate a well-informed answer. It’s like having a knowledgeable assistant who looks up info and then gives you a clear, concise response.

Why Use RAG?
More Accurate Responses: By pulling in specific, relevant information from external documents, RAG can provide more accurate and detailed answers than a model relying solely on what it “learned” during training.
Keeps Up with New Information: RAG can dynamically retrieve up-to-date information, which is super helpful for topics that change frequently.

Reduces Errors: Pure generative models sometimes “make stuff up.” RAG minimizes this by grounding answers in real, retrieved documents.

Where You’ll See RAG in Action
Customer Support: Chatbots that use RAG can pull from a company's knowledge base to provide accurate, real-time support.
Research Assistants: Assists with finding and summarizing relevant articles or papers.
Open-Domain Q&A: Perfect for answering questions that could have answers across various domains or sources.

The Bottom Line

RAG combines the best of both worlds: it uses powerful retrieval techniques to find the best supporting documents and then leverages the generative capabilities of language models to craft coherent, informed answers. It’s a smart approach for any task where you want accurate, context-rich responses.

**Next we will be using Chroma DB to store vectorised documents and also we will use SimpleSequentialChain**

In [4]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Create Sample Text Documents as LangChain Documents
sample_texts = [
    "Washington, D.C., formally the District of Columbia, is the capital city of the United States. It is located on the east bank of the Potomac River and was founded after the American Revolution to serve as the nation's capital.",
    "The United States of America (USA) is a country primarily located in North America. It consists of 50 states, a federal district, and several territories.",
    "San Francisco is a major city in California, known for its Golden Gate Bridge, cable cars, and colorful Victorian houses.",
    "Harry Potter is a series of seven fantasy novels written by British author J.K. Rowling. The series chronicles the lives of a young wizard, Harry Potter, and his friends as they attend Hogwarts School of Witchcraft and Wizardry."
]

# Convert sample texts into LangChain Document objects
documents = [Document(page_content=text) for text in sample_texts]

# Chunk Documents
# Split documents into chunks for better retrieval performance
text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=50)
chunked_documents = text_splitter.split_documents(documents)

# Index the Documents using ChromaDB
# Use Hugging Face embeddings for indexing
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Initialize ChromaDB vector store
chroma_db = Chroma.from_documents(documents=chunked_documents, embedding=embedding_model)

# Create a retriever from ChromaDB
retriever = chroma_db.as_retriever()

#  Set Up the RAG Pipeline
# Choose and load a Hugging Face model for text generation
model_name = "google/flan-t5-large"  # Model suitable for text-to-text generation tasks
hf_pipeline = pipeline("text2text-generation", model=model_name, clean_up_tokenization_spaces=True)  # Set to avoid FutureWarning

# Wrap the Hugging Face pipeline using LangChain's HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=hf_pipeline)

# Create a PromptTemplate for generating responses
prompt_template = PromptTemplate(input_variables=["context", "question"], template="{context}\n\n{question}")

# Chain Setup: Chain should properly retrieve and pass context
def retrieve_and_format(question):
    retrieved_docs = retriever.get_relevant_documents(question)
    context = "\n".join([doc.page_content for doc in retrieved_docs])
    return {"context": context, "question": question}

# Setup the pipeline correctly using components
chain = (
    RunnablePassthrough()  # This will pass the input question to the next step
    | retrieve_and_format  # Retrieve and format documents to create context
    | prompt_template  # Use the prompt template to structure the input
    | llm  # Generate a response using the language model
    | StrOutputParser() # Parse the output as a string
)

# Run the Chain Pipeline with the New Question
question = "Who is the author of Harry Potter?"
response = chain.invoke(question)
print(response)

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:151: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed i

J.K. Rowling


**Concept of Memory** 

In [5]:
from langchain.memory import ConversationBufferMemory

In [6]:
memory1 = ConversationBufferMemory( memory_key="history") 

memory1.save_context({"input": "hey"}, {"output": "wassup"}) 

print(f"memory1 ==> {memory1.load_memory_variables({})}")

memory2 = ConversationBufferMemory( memory_key="chat_history") 

memory2.save_context({"input": "hi"}, {"output": "whats are you doing?"}) 

print(f"memory2 ==> {memory2.load_memory_variables({})}")

memory1 ==> {'history': 'Human: hey\nAI: wassup'}
memory2 ==> {'chat_history': 'Human: hi\nAI: whats are you doing?'}


In [7]:



# Load the Hugging Face LLM model
hf_pipeline = pipeline('text-generation', model='gpt2', max_length=256, max_new_tokens=50)

# Wrap the pipeline in a HuggingFacePipeline for compatibility with LangChain
llm = HuggingFacePipeline(pipeline=hf_pipeline)

# Set up the embedding model for vector storage
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Sample documents for testing
documents = [
    Document(page_content="Paris is the capital of France."),
    Document(page_content="France is a country in Europe."),
    Document(page_content="The French Revolution was a period of social and political upheaval in France.")
]

#  Embed the documents
embedded_docs = [(doc, embedding_model.embed_query(doc.page_content)) for doc in documents]
doc_texts = [doc.page_content for doc in documents]
embeddings = [embedding_model.embed_query(text) for text in doc_texts]

#  Initialize the FAISS index
dimension = len(embeddings[0])  # Assuming all embeddings have the same length
index = faiss.IndexFlatL2(dimension)  # Using L2 (Euclidean) distance
index.add(np.array(embeddings))

# Initialize the FAISS vector store with InMemoryDocstore
docstore = InMemoryDocstore(dict(enumerate(documents)))
vector_store = FAISS(embedding_function=embedding_model, index=index, docstore=docstore, index_to_docstore_id={i: i for i in range(len(documents))})

# Initialize memory buffer with explicit output key
memory = ConversationBufferMemory(output_key='result')  # Set output_key to 'result'




config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
#  Create a retrieval-based QA chain
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 2})

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

# Attach memory buffer to the chain
qa_chain.memory = memory

# Define a function to use the QA model with memory buffer
def query_with_memory(query: str):
    # Run the retrieval-based QA chain and get the results
    result = qa_chain({"query": query})
    
    # Extract the desired output ('result') to save in memory
    output_for_memory = {"result": result['result']}
    
    # Save only the relevant output key to memory
    memory.save_context({"query": query}, output_for_memory)
    
    # Retrieve memory for debugging or review
    memory_content = memory.load_memory_variables({})
    
    print("Memory Buffer Content:", memory_content)
    return result


In [9]:

# Test the implementation
query = "What is the capital of France?"
response = query_with_memory(query)
print("Response:", response['result'])  # Use 'result' to match the correct key

# Add more queries to see memory buffer in action
query = "Tell me more about its history."
response = query_with_memory(query)
print("Response:", response['result'])  # Use 'result' to match the correct key

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:151: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=50) and `max_length`(=256) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

Memory Buffer Content: {'history': "Human: What is the capital of France?\nAI: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nParis is the capital of France.\n\nFrance is a country in Europe.\n\nQuestion: What is the capital of France?\nHelpful Answer: Capital Paris is located in eastern Paris. It is home to one of Europe's largest cities, Le Bourget. French nationals with French passports at the airport can take up to 10 flights a day to Paris. This ensures they are safe and connected to\nHuman: What is the capital of France?\nAI: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nParis is the capital of France.\n\nFrance is a country in Europe.\n\nQuestion: What is the capital of France?\nHelpful Answer: Capital Paris is located in eastern Paris. It is h